In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno

from utilities import clean_data, missing

In [95]:
df, df_brands, df_allbrands, brands, compsets, compset_groups, groups_bycompset = clean_data()

print(df.shape)
df.head(n=5)

(298040, 7)


,period_end_date,business_entity_doing_business_as_name,followers,pictures,videos,comments,likes
0,2017-05-06,24S,NaN,NaN,NaN,NaN,NaN
1,2017-05-13,24S,NaN,6.0,3.0,57.0,1765.0
2,2017-05-20,24S,NaN,6.0,3.0,57.0,1765.0
3,2017-05-27,24S,NaN,6.0,3.0,57.0,1765.0
4,2017-06-03,24S,NaN,24.0,3.0,109.0,3922.0


# 1) Understanding Missing Values

In [96]:


# get the number of missing data points per column
missing_values_count = df.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[::]

period_end_date                               0
business_entity_doing_business_as_name        0
followers                                 61727
pictures                                   3864
videos                                     9602
comments                                   4082
likes                                      3752
dtype: int64

In [97]:
df_missing = missing(df)

df_missing.head()

,business_entity_doing_business_as_name,followers_NaNs,pictures_NaNs,videos_NaNs,comments_NaNs,likes_NaNs
0,24S,20,1,1,1,1
1,3.1 Phillip Lim,18,0,0,0,0
2,3CE,131,0,0,0,0
3,A. Lange & Soehne,8,3,2,2,2
4,ANIMALE,131,0,2,0,0


#### How many rows are in df, containing at least one Nan?

In [98]:
# Count the number of rows with at least one NaN value across any column and print
rows_with_nan = df.isna().any(axis=1).sum()
print("Number of rows with at least one NaN:", rows_with_nan)

Number of rows with at least one NaN: 65868


In [99]:
#df[0:50]
#df[300:350]

#### Removing all rows at the beginning of each company that are part of a series of Nan entries in the followers column:

In [100]:
# Function to remove leading NaNs in 'followers' for each group(business)
def remove_leading_nans(group):
    # Get the first index where 'followers' is not NaN
    first_valid_index = group['followers'].first_valid_index()
    # If all are NaNs, return an empty DataFrame from this group
    if first_valid_index is None:
        return pd.DataFrame()
    # Return the DataFrame starting from the first non-NaN 'followers' row
    return group.loc[first_valid_index:]

# Apply the function to each group and concatenate the results
cleaned_df = df.groupby('business_entity_doing_business_as_name', group_keys=False).apply(remove_leading_nans)


In [101]:

# Count the number of rows with at least one NaN value across any column
rows_with_nan = cleaned_df.isna().any(axis=1).sum()

# Display the count
print("Number of rows with at least one NaN:", rows_with_nan)

Number of rows with at least one NaN: 4378


In [102]:
#cleaned_df[0:50]

#cleaned_df[1100:1150]

#### How long are the remaining series of Nan's in each column?

In [103]:
cleaned_df = cleaned_df.reset_index(drop=True)

In [104]:
def calculate_nan_series_lengths_indices_and_total(_df):
    # Initialize a dictionary to store the lengths of NaN series and their indices for each column
    nan_series_details = {col: [] for col in _df.columns if _df[col].isna().any()}
    total_nan_count = 0
    
    # Iterate through each column that contains NaN
    for col in nan_series_details.keys():
        current_series_length = 0
        series_start_index = None
        previous_business = None
        
        # Iterate through each row
        for idx, row in _df.iterrows():
            if pd.isna(row[col]):
                # Check if we've moved to a new business
                if row['business_entity_doing_business_as_name'] != previous_business:
                    if current_series_length > 0:
                        # Save the length and indices of the previous series before starting a new one
                        nan_series_details[col].append((current_series_length, series_start_index, idx - 1))
                        total_nan_count += current_series_length
                    # Reset the series length for the new business and record start index
                    current_series_length = 1
                    series_start_index = idx
                else:
                    # Increment the series length
                    current_series_length += 1
            else:
                if current_series_length > 0:
                    # End of a series, append its length and indices to the list and add to total count
                    nan_series_details[col].append((current_series_length, series_start_index, idx - 1))
                    total_nan_count += current_series_length
                    current_series_length = 0
                    series_start_index = None
            
            # Update the previous business name
            previous_business = row['business_entity_doing_business_as_name']
        
        # Check if the last row in the dataframe was a NaN and needs to be added
        if current_series_length > 0:
            nan_series_details[col].append((current_series_length, series_start_index, _df.index[-1]))
            total_nan_count += current_series_length

    return nan_series_details, total_nan_count

nan_series_info, total_nan = calculate_nan_series_lengths_indices_and_total(cleaned_df)
print(nan_series_info)
print("Total NaN entries:", total_nan)

{'followers': [(74, None, 20842), (43, None, 30082), (89, None, 34146), (31, None, 71309)], 'pictures': [(3, 6061, 6063), (1, 7958, 7958), (53, 16635, 16687), (19, 20421, 20439), (96, 21483, 21578), (53, 29628, 29680), (1, 43464, 43464), (52, 49020, 49071), (109, 52645, 52753), (21, 58280, 58300), (73, 59124, 59196), (37, 72337, 72373), (58, 73044, 73101), (77, 73877, 73953), (11, 77736, 77746), (1, 78063, 78063), (22, 79482, 79503), (53, 85486, 85538), (35, 89124, 89158), (53, 91502, 91554), (52, 93247, 93298), (39, 95034, 95072), (42, 99706, 99747), (11, 100670, 100680), (53, 101828, 101880), (18, 105975, 105992), (136, 106980, 107115), (52, 107734, 107785), (27, 108189, 108215), (27, 108618, 108644), (50, 110924, 110973), (35, 114638, 114672), (52, 119911, 119962), (94, 124339, 124432), (174, 124766, 124939), (52, 129233, 129284), (118, 132922, 133039), (52, 137185, 137236), (25, 137929, 137953), (172, 140454, 140625), (47, 152086, 152132), (23, 153856, 153878), (28, 156032, 156059)

In [105]:
#total_nan_entries = df.isna().sum().sum()

print(cleaned_df.isna().sum().sum())

14027


In [106]:
#'followers': [(74, 20769, 20842), (43, 30040, 30082), (89, 34058, 34146), (31, 71279, 71309)]

#cleaned_df[71309-31-1:71309+5]

#drop above rows manually:

indices_to_drop = list(range(20769, 20842)) + list(range(30040, 30082)) + list(range(34058, 34146)) + list(range(71279, 71309))
cleaned_df = cleaned_df.drop(indices_to_drop)


In [107]:
cleaned_df[cleaned_df['followers'].isna()]

,period_end_date,business_entity_doing_business_as_name,followers,pictures,videos,comments,likes
20842,2023-09-16,Becca,NaN,0.0,0.0,0.0,0.0
30082,2023-09-16,Bottega Veneta,NaN,0.0,0.0,0.0,0.0
34146,2023-09-16,Bulgari Beauty,NaN,0.0,0.0,0.0,0.0
71309,2023-09-16,East Bay,NaN,0.0,0.0,0.0,0.0


#### How many rows with 4, 3, 2 nans are there?

In [108]:
# Calculate the number of NaNs per row
nan_counts_per_row = cleaned_df.isna().sum(axis=1)

# Count how many rows have exactly 4, 3, and 2 NaNs
nan_4 = (nan_counts_per_row == 4).sum()
nan_3 = (nan_counts_per_row == 3).sum()
nan_2 = (nan_counts_per_row == 2).sum()
nan_1 = (nan_counts_per_row == 1).sum()

print("Number of rows with 4 NaNs:", nan_4)
print("Number of rows with 3 NaNs:", nan_3)
print("Number of rows with 2 NaNs:", nan_2)
print("Number of rows with 1 NaNs:", nan_1)

# For a more general approach, to see counts for all possible numbers of NaNs
nan_counts_distribution = nan_counts_per_row.value_counts().sort_index()

print("Distribution of NaN counts per row:\n", nan_counts_distribution)

print(nan_1+nan_2+nan_3+nan_4)
print(nan_1+2*nan_2+3*nan_3+4*nan_4)

Number of rows with 4 NaNs: 3214
Number of rows with 3 NaNs: 0
Number of rows with 2 NaNs: 7
Number of rows with 1 NaNs: 924
Distribution of NaN counts per row:
 0    232172
1       924
2         7
4      3214
dtype: int64
4145
13794


In [110]:
print(cleaned_df.shape)



(236317, 7)

# 2) Calculate rate of change of followers and cross correlate

In [109]:
df[300:350]
#print(df.shape)
#print(df[df['pictures'].isna()].shape)

,period_end_date,business_entity_doing_business_as_name,followers,pictures,videos,comments,likes
300,2023-02-04,24S,214756.0,18.0,9.0,249.0,31076.0
301,2023-02-11,24S,214643.0,18.0,9.0,191.0,21015.0
302,2023-02-18,24S,214925.0,19.0,9.0,592.0,25620.0
303,2023-02-25,24S,214665.0,20.0,8.0,591.0,25879.0
304,2023-03-04,24S,214590.0,19.0,11.0,600.0,15842.0
305,2023-03-11,24S,214519.0,18.0,11.0,574.0,14832.0
306,2023-03-18,24S,214323.0,17.0,12.0,196.0,11378.0
307,2023-03-25,24S,214605.0,17.0,12.0,226.0,12650.0
308,2023-04-01,24S,214625.0,19.0,8.0,191.0,10187.0
309,2023-04-08,24S,214684.0,20.0,9.0,210.0,13820.0
